In [2]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

import tensorflow as tf
import tensorflow.keras as keras
from alibi.explainers import Counterfactual
from alibi.explainers import CounterfactualRLTabular, CounterfactualRL
from alibi.explainers import CounterfactualProto
#from alibi.datasets import fetch_adult
from alibi.models.tensorflow import HeAE
from alibi.models.tensorflow import Actor, Critic
#from alibi.models.tensorflow import ADULTEncoder, ADULTDecoder
from alibi.explainers.cfrl_base import Callback
from alibi.explainers.backends.cfrl_tabular import get_he_preprocessor, get_statistics, \
    get_conditional_vector, apply_category_mapping


In [67]:
credit_data = pd.read_csv("datasets/Credit.csv",index_col=0)

In [9]:
from sklearn.preprocessing import OrdinalEncoder

# List of categorical features
cat_features = [
    'checkingstatus1', 'history', 'purpose', 'savings', 'employ',
    'sex', 'housing', 'tele', 'foreign'
]

# Initialize ordinal encoder
encoder = OrdinalEncoder()
credit_data[cat_features] = encoder.fit_transform(credit_data[cat_features])

In [10]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X = credit_data.drop(columns=['Default'])
y = credit_data['Default']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape

((800, 16), (200, 16))

In [11]:
import xgboost as xgb

# Define the XGBoost classifier
clf = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    n_estimators=100
)

# Train the model
clf.fit(X_train, y_train)

`use_label_encoder` is deprecated in 1.7.0.


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None, ...)

In [12]:
from sklearn.metrics import f1_score, classification_report

# Make predictions on the test set
y_pred = clf.predict(X_test)

print(f"F1 Score: {f1_score(y_test, y_pred)}")

print(classification_report(y_test, y_pred))

F1 Score: 0.5306122448979591
              precision    recall  f1-score   support

           0       0.80      0.91      0.85       141
           1       0.67      0.44      0.53        59

    accuracy                           0.77       200
   macro avg       0.73      0.67      0.69       200
weighted avg       0.76      0.77      0.75       200



In [19]:
def predict_fn(X):
    # The predict_proba method of the pipeline returns an array of shape (n_samples, 2)
    # Return both columns as the CounterfactualProto explainer expects a probability for each class
    pred_proba = clf.predict_proba(X)
    return np.hstack([1 - pred_proba[:, 1].reshape(-1, 1), pred_proba[:, 1].reshape(-1, 1)])



In [20]:
# Calculate the min and max values for the entire training dataset
feature_min = np.min(X_train, axis=0)
feature_max = np.max(X_train, axis=0)

In [21]:
from alibi.explainers import CounterfactualProto

tf.compat.v1.disable_eager_execution()

# Initialize CounterFactualProto
shape = (1,) + X_train.shape[1:]
# Initialize the explainer
cf = CounterfactualProto(predict_fn, shape, use_kdtree=True, theta=10., max_iterations=1000,
                         feature_range=(feature_min, feature_max), 
                         c_init=1., c_steps=10)
cf.fit(X_train.values)

No encoder specified. Using k-d trees to represent class prototypes.


CounterfactualProto(meta={
  'name': 'CounterfactualProto',
  'type': ['blackbox', 'tensorflow', 'keras'],
  'explanations': ['local'],
  'params': {
              'kappa': 0.0,
              'beta': 0.1,
              'gamma': 0.0,
              'theta': 10.0,
              'cat_vars': None,
              'ohe': False,
              'use_kdtree': True,
              'learning_rate_init': 0.01,
              'max_iterations': 1000,
              'c_init': 1.0,
              'c_steps': 10,
              'eps': (0.001, 0.001),
              'clip': (-1000.0, 1000.0),
              'update_num_grad': 1,
              'write_dir': None,
              'feature_range': (checkingstatus1      0.0
duration             4.0
history              0.0
purpose              0.0
amount             250.0
savings              0.0
employ               0.0
installment          1.0
sex                  0.0
residence            1.0
age                 19.0
housing              0.0
cards                1.0
li

In [29]:
instances = X_test[y_pred == 1].values

explanation = cf.explain(instances[1].reshape(1, -1))
explanation.cf['X']

# # Generate counterfactuals
# counterfactuals = []
# for instance in instances:
#     explanation = cf.explain(instance.reshape(1, -1))
#     counterfactuals.append(explanation.cf['X'])

# Generate the counterfactuals
# counterfactuals = []

# for instance in instances:
#     explanation = cf.explain(instance.reshape(1, -1))
#     if explanation.cf is not None and 'X' in explanation.cf:
#         counterfactuals.append(explanation.cf['X'])
#     else:
#         counterfactuals.append(None)

# # Count the number of instances for which no counterfactual was found
# no_cf_count = counterfactuals.count(None)

# no_cf_count


array([[  1.      ,  10.465166,   1.      ,   2.      , 585.62634 ,
          0.      ,   1.      ,   4.      ,   1.      ,   4.      ,
         21.534851,   2.      ,   2.      ,   1.      ,   0.      ,
          0.      ]], dtype=float32)

In [42]:
instances[1]

array([  1.,  12.,   1.,   2., 585.,   0.,   1.,   4.,   1.,   4.,  20.,
         2.,   2.,   1.,   0.,   0.])

In [43]:
explanation0 = cf.explain(instances[0].reshape(1, -1))
print(explanation0.cf['X'])
print(instances[0])

[[1.5681556e+00 1.9935682e+01 0.0000000e+00 3.0000000e+00 2.3218469e+03
  1.0000000e+00 1.4279537e+00 2.0000000e+00 1.0000000e+00 3.0000000e+00
  3.2427963e+01 1.0000000e+00 1.0000000e+00 1.0000000e+00 0.0000000e+00
  0.0000000e+00]]
[0.000e+00 2.400e+01 0.000e+00 3.000e+00 2.325e+03 1.000e+00 1.000e+00
 2.000e+00 1.000e+00 3.000e+00 3.200e+01 1.000e+00 1.000e+00 1.000e+00
 0.000e+00 0.000e+00]


In [44]:
explanation2 = cf.explain(instances[2].reshape(1, -1))
print(explanation2.cf['X'])
print(instances[2])

[[0.0000000e+00 3.8087887e+01 1.0000000e+00 2.0000000e+00 3.9610884e+03
  0.0000000e+00 1.7780616e+00 4.0000000e+00 1.0000000e+00 3.0000000e+00
  3.1510588e+01 1.0000000e+00 1.0000000e+00 1.0000000e+00 3.5536310e-01
  0.0000000e+00]]
[0.000e+00 4.200e+01 1.000e+00 2.000e+00 3.965e+03 0.000e+00 0.000e+00
 4.000e+00 1.000e+00 3.000e+00 3.400e+01 1.000e+00 1.000e+00 1.000e+00
 0.000e+00 0.000e+00]


In [45]:
explanation3 = cf.explain(instances[3].reshape(1, -1))
print(explanation3.cf['X'])
print(instances[3])

[[5.0000006e-01 5.7699997e+01 1.0000000e+00 3.0000004e-01 7.2871001e+03
  0.0000000e+00 2.0000000e+00 3.8999999e+00 1.0000000e+00 3.8999999e+00
  3.5299999e+01 1.9000000e+00 1.1000000e+00 1.1000000e+00 0.0000000e+00
  0.0000000e+00]]
[0.000e+00 6.000e+01 1.000e+00 0.000e+00 7.297e+03 0.000e+00 2.000e+00
 4.000e+00 1.000e+00 4.000e+00 3.600e+01 2.000e+00 1.000e+00 1.000e+00
 0.000e+00 0.000e+00]


In [ ]:
# need a decoder! 

# DiCE Random

In [40]:
continuous_features = ['duration', 'amount', 'installment', 'residence', 'age', 'cards', 'liable']
categorical_features = ['checkingstatus1', 'history', 'purpose', 'savings', 'employ', 'sex', 'housing', 'tele', 'foreign']
immutable_features = ['sex', 'liable', 'foreign', 'purpose']
non_decreasing_features = ['age', 'employ']
correlated_features = []

In [41]:
import dice_ml
import time
from dice_ml.utils import helpers

full_train_df = pd.DataFrame(X_train, columns=X.columns)

# Add the target column
full_train_df['target'] = y_train.values

# Prepare a random instance from the test set for which you want to generate counterfactuals
instances_1 = X_test[y_pred == 1]
query_instance = instances_1.sample(1)
query_instance_df = pd.DataFrame(query_instance, columns=X.columns)


# Define continuous features (assuming all non-categorical columns are continuous)
continuous_features = [col for col in X_train.columns if col not in cat_features + ['target']]

# Create the data object for DiCE
d = dice_ml.Data(dataframe=full_train_df, continuous_features=continuous_features, outcome_name='target')
backend = 'sklearn'
method = 'random'

class ModelWrapper:
    def __init__(self, model):
        self.model = model
    
    def predict_proba(self, data):
        if isinstance(data, pd.DataFrame):
            return self.model(data.values)
        else:
            return self.model(data)

def model_predict(data):
    if isinstance(data, pd.DataFrame):
        return clf.predict_proba(data.values)
    else:
        return clf.predict_proba(data)

# 使用包装器
wrapped_model = ModelWrapper(model_predict)
m = dice_ml.Model(model=wrapped_model, backend=backend)


# Create the DiCE explainer and generate counterfactuals
exp = dice_ml.Dice(d, m, method=method)


# Loop through each instance in instances_1 and generate counterfactuals
counterfactuals_list = []
for _, instance in instances_1.iterrows():
    instance_df = pd.DataFrame(instance).T
    dice_exp = exp.generate_counterfactuals(instance_df, total_CFs=2, desired_class="opposite")
    cf_df = dice_exp.visualize_as_dataframe(show_only_changes=True)
    counterfactuals_list.append((instance, cf_df))



UserConfigValidationException will be deprecated from dice_ml.utils. Please import UserConfigValidationException from raiutils.exceptions.
100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.12it/s]

Query instance (original outcome : 1)


,checkingstatus1,duration,history,purpose,amount,savings,employ,installment,sex,residence,age,housing,cards,liable,tele,foreign,target
0,0.0,24,0.0,3.0,2325,1.0,1.0,2,1.0,3,32,1.0,1,1,0.0,0.0,1



Diverse Counterfactual set (new outcome: 0.0)


,checkingstatus1,duration,history,purpose,amount,savings,employ,installment,sex,residence,age,housing,cards,liable,tele,foreign,target
0,0.0,-,0.0,3.0,-,1.0,1.0,-,1.0,-,52.0,1.0,-,-,1.0,0.0,0
1,0.0,35.0,0.0,3.0,-,1.0,1.0,-,1.0,-,-,1.0,-,-,0.0,0.0,0


...

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.14it/s]

Query instance (original outcome : 1)


,checkingstatus1,duration,history,purpose,amount,savings,employ,installment,sex,residence,age,housing,cards,liable,tele,foreign,target
0,0.0,24,0.0,4.0,3632,0.0,1.0,1,0.0,4,22,2.0,1,1,0.0,1.0,1



Diverse Counterfactual set (new outcome: 0.0)


,checkingstatus1,duration,history,purpose,amount,savings,employ,installment,sex,residence,age,housing,cards,liable,tele,foreign,target
0,0.0,31.0,0.0,4.0,-,0.0,1.0,-,0.0,-,-,-,-,-,0.0,1.0,0
1,0.0,40.0,0.0,4.0,-,0.0,1.0,-,0.0,-,-,2.0,-,-,0.0,1.0,0


# Alibi RL

In [72]:
import tensorflow as tf
from tensorflow import keras

# 获取分类特征
categorical_features = credit_data.select_dtypes(include=['object']).columns.tolist()
# 获取数值特征
numerical_features = credit_data.select_dtypes(include=['float64', 'int64']).columns.tolist()
numerical_features.remove('Default')

# 定义数值型特征的输入
numerical_inputs = keras.Input(shape=(len(numerical_features),), name="numerical")

# 定义分类型特征的输入
categorical_inputs = [keras.Input(shape=(1,), name=cat_feature) for cat_feature in categorical_features]

# 编码器部分

categorical_vocabs = [credit_data[cat_feature].nunique() for cat_feature in categorical_features]
encoded_numerical = keras.layers.Dense(64, activation="relu")(numerical_inputs)
encoded_categorical = [keras.layers.Embedding(input_dim=cat_vocab, output_dim=8)(cat_input) for cat_input, cat_vocab in zip(categorical_inputs, categorical_vocabs)]
encoded_categorical = [keras.layers.Flatten()(embed) for embed in encoded_categorical]

latent_dim = 8
# 合并编码后的特征
encoded_features = keras.layers.Concatenate()([encoded_numerical] + encoded_categorical)
latent_space = keras.layers.Dense(latent_dim, activation="relu")(encoded_features)

# 解码器部分
decoded_numerical = keras.layers.Dense(len(numerical_features), activation="linear")(latent_space)
decoded_categorical = [keras.layers.Dense(cat_vocab, activation="softmax")(latent_space) for cat_vocab in categorical_vocabs]

# 构建自编码器模型
encoder = keras.Model(inputs=[numerical_inputs] + categorical_inputs, outputs=latent_space)

# 定义解码器的输入层
decoder_input = keras.Input(shape=(latent_dim,), name="decoder_input")

# 使用解码器的层将新的输入层解码回原始空间
decoded_numerical_output = keras.layers.Dense(len(numerical_features), activation="linear")(decoder_input)
decoded_categorical_outputs = [keras.layers.Dense(vocab_size, activation="softmax")(decoder_input) for vocab_size in categorical_vocabs]

# 定义解码器模型
decoder = keras.Model(inputs=decoder_input, outputs=[decoded_numerical_output] + decoded_categorical_outputs)

# 使用编码器和解码器定义完整的自编码器模型
autoencoder_outputs = decoder(encoder([numerical_inputs] + categorical_inputs))
autoencoder = keras.Model(inputs=[numerical_inputs] + categorical_inputs, outputs=autoencoder_outputs)

# 编译模型
autoencoder.compile(optimizer="adam", loss=["mse"] + ["categorical_crossentropy"] * len(categorical_features))


In [73]:
categorical_features

['checkingstatus1',
 'history',
 'purpose',
 'savings',
 'employ',
 'sex',
 'housing',
 'tele',
 'foreign']

In [74]:
numerical_features

['duration', 'amount', 'installment', 'residence', 'age', 'cards', 'liable']

In [76]:
# 假设您的数据是X_test

# 将X_test分解为多个数组
inputs = [X_test[numerical_features].values] + [X_test[cat_feature].values for cat_feature in categorical_features]

# 使用编码器得到编码结果
encoded_data = encoder.predict(inputs)

# 使用解码器得到解码结果
decoded_data = decoder.predict(encoded_data)



`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.


In [77]:
encoded_data

array([[ 256.25372,    0.     ,  185.05359, ...,  524.71814,    0.     ,
           0.     ],
       [ 354.11032,    0.     ,  252.47156, ...,  719.7493 ,    0.     ,
           0.     ],
       [ 185.64175,    0.     ,  137.77817, ...,  383.2987 ,    0.     ,
           0.     ],
       ...,
       [ 527.29706,    0.     ,  376.88644, ..., 1079.7372 ,    0.     ,
           0.     ],
       [ 289.56705,    0.     ,  211.16765, ...,  598.47314,    0.     ,
           0.     ],
       [ 753.0077 ,    0.     ,  545.914  , ..., 1553.1106 ,    0.     ,
           0.     ]], dtype=float32)

In [78]:
decoded_data

[array([[   71.785416,  -697.51227 ,  -203.29886 , ...,   428.89395 ,
           388.27206 ,  -184.90216 ],
        [   99.20991 ,  -957.3672  ,  -280.3802  , ...,   588.46484 ,
           532.44763 ,  -252.9188  ],
        [   49.776978,  -509.32928 ,  -147.98804 , ...,   312.65475 ,
           281.13812 ,  -135.55675 ],
        ...,
        [  151.99545 , -1434.662   ,  -417.4381  , ...,   883.26245 ,
           804.02075 ,  -380.4143  ],
        [   81.60968 ,  -794.2169  ,  -230.2409  , ...,   488.41876 ,
           443.81995 ,  -211.54353 ],
        [  215.46242 , -2062.2449  ,  -597.43054 , ...,  1269.4846  ,
          1155.0159  ,  -548.77234 ]], dtype=float32),
 array([[7.49851418e-24, 7.90684752e-24, 1.00000000e+00, 0.00000000e+00],
        [1.52097052e-32, 1.49867007e-32, 1.00000000e+00, 0.00000000e+00],
        [3.13897548e-18, 4.03672742e-17, 1.00000000e+00, 0.00000000e+00],
        [1.09062682e-10, 9.33660482e-10, 1.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0

In [ ]:
explainerrl = CounterfactualRLTabular(predictor=predictor,
                                    encoder=heae.encoder,
                                    decoder=heae.decoder,
                                    latent_dim=LATENT_DIM,
                                    encoder_preprocessor=heae_preprocessor,
                                    decoder_inv_preprocessor=heae_inv_preprocessor,
                                    coeff_sparsity=COEFF_SPARSITY,
                                    coeff_consistency=COEFF_CONSISTENCY,
                                    category_map=category_map,
                                    feature_names=feature_names,
                                    ranges=ranges,
                                    immutable_features=immutable_features,
                                    train_steps=TRAIN_STEPS,
                                    batch_size=BATCH_SIZE,
                                    backend="tensorflow")